# Praktikum 2 : Generator Teks dengan RNN

## Setup

**Import TensorFlow**

In [51]:
# Mengimpor TensorFlow, framework untuk machine learning dan deep learning.
import tensorflow as tf

# Mengimpor NumPy, library yang sering digunakan untuk operasi numerik.
import numpy as np

# Mengimpor modul os untuk berinteraksi dengan sistem operasi.
import os

# Mengimpor modul time untuk mengukur waktu eksekusi.
import time

**Download Dataset Shakespeare**

In [52]:
# Mengunduh file 'shakespeare.txt' dari URL yang diberikan dan menyimpannya di path yang ditentukan.
path_to_file=tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

**Load Data**

In [53]:
# Membaca file teks 'shakespeare.txt' dalam mode baca biner (rb), kemudian mendekode teksnya menggunakan utf-8.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# Menampilkan panjang teks yang telah dibaca dalam jumlah karakter.
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [54]:
# Menampilkan 250 karakter pertama dari teks.
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [55]:
# Mencari karakter-karakter unik dalam teks dan mengurutkannya.
vocab = sorted(set(text))
# Menampilkan jumlah karakter unik yang ada dalam teks.
print(f'{len(vocab)} unique characters')

65 unique characters


## Olah Teks
**Vectorize Tekz**

In [56]:
# Membuat daftar teks contoh yang berisi dua string.
example_texts=['abcdefg','xyz']
# Menggunakan tf.strings.unicode_split untuk memecah teks menjadi karakter unicode.
chars=tf.strings.unicode_split(example_texts,input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [57]:
# Membuat kamus (lookup) untuk menghubungkan karakter dengan ID numerik.
ids_from_chars=tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [58]:

# Mengonversi karakter-karakter dalam variabel `chars` menjadi ID numerik.
ids=ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [59]:
# Membuat kamus (lookup) untuk menghubungkan ID numerik kembali ke karakter.
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [60]:
# Mengonversi ID numerik kembali menjadi karakter.
chars=chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [61]:
# Menggabungkan karakter-karakter dalam `chars` menjadi string tunggal.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [62]:
# Mendefinisikan fungsi untuk mengonversi ID numerik menjadi teks.
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# Prediksi
**Membuat Training Set dan Target**

In [63]:
# Mengonversi seluruh teks menjadi urutan ID numerik.
all_ids=ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [64]:
# Membuat dataset TensorFlow dari tensor ID numerik.
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [65]:
# Mengambil 10 urutan karakter dari dataset dan mengonversi kembali ke teks.
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [66]:
# Mengelompokkan urutan ID numerik menjadi urutan yang lebih pendek dengan panjang tertentu.
seq_length = 100

In [67]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
# Menampilkan satu urutan yang telah dibagi.
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [68]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [69]:
# Menggunakan fungsi split_input_target untuk memisahkan input dan target.
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [70]:
# mengilustrasikan bagaimana fungsi split_input_target bekerja.
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [71]:
# Mengaplikasikan fungsi split_input_target ke semua urutan dalam dataset.
dataset = sequences.map(split_input_target)

In [72]:
# Menampilkan contoh input dan target dari dataset.
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


**Membuat  Batch Training**

In [73]:
# Menentukan BATCH_SIZE (ukuran batch) untuk pelatihan model.
BATCH_SIZE = 64

# Menentukan BUFFER_SIZE untuk proses pengacakan dataset.
BUFFER_SIZE = 10000

# Menyiapkan dataset dengan langkah-langkah berikut:
# 1. Melakukan pengacakan elemen-elemen dataset menggunakan BUFFER_SIZE.
# 2. Mengelompokkan dataset menjadi batch dengan ukuran BATCH_SIZE dan menjatuhkan sisa jika ada.
# 3. Menggunakan prefetch untuk mengoptimalkan proses pembacaan data.
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Membuat Model

In [74]:
# Menentukan ukuran vocab, dimensi embedding, dan jumlah unit RNN dalam model.

# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [75]:
# Mendefinisikan kelas MyModel yang akan digunakan sebagai model untuk pemrosesan teks.

# Kode ini mendefinisikan kelas MyModel yang merupakan turunan dari tf.keras.Model. 
# Model ini terdiri dari beberapa lapisan, termasuk lapisan embedding (untuk merepresentasikan karakter dalam vektor), lapisan GRU (Recurrent Unit Gate) yang digunakan untuk memproses urutan, dan lapisan dense yang digunakan untuk menghasilkan output.
# Model ini memiliki metode call, yang digunakan untuk menggambarkan aliran data melalui model. Model ini memiliki fleksibilitas dalam mengembalikan output dan/atau status internal.

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [76]:
# Membuat model MyModel dengan parameter yang telah ditentukan sebelumnya.
# membuat model MyModel dengan parameter-parameter yang telah ditentukan sebelumnya, seperti ukuran vocab, dimensi embedding, dan jumlah unit RNN. 
# Model ini siap untuk digunakan dalam pelatihan atau pengujian.
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Uji Model

In [77]:
# Mengambil contoh batch input dan target dari dataset.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [78]:
# Menampilkan ringkasan (summary) dari model.
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  16896     
                                                                 
 gru_2 (GRU)                 multiple                  3938304   
                                                                 
 dense_2 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [79]:
# Mengambil indeks karakter yang diambil secara acak dari prediksi model.
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [80]:
# Menampilkan index karakter yang diambil secara acak
sampled_indices

array([57, 48,  6, 58, 39,  1,  4, 24,  6, 53, 12,  9, 54, 64, 56, 33, 30,
       21, 56, 26, 35, 31, 31, 33, 41, 48, 58, 42, 22, 43,  1, 28, 55, 54,
       42, 13,  1, 44, 59,  2, 16, 48, 65, 31, 62,  5, 56, 20, 35, 46,  5,
       39, 13, 43, 54, 20, 64, 54, 14,  7, 65, 34,  7, 48, 30, 16, 24, 14,
        3, 21,  1, 49, 10, 64, 45,  9,  4,  3, 24, 42,  1, 28, 30,  3, 18,
       38, 50, 23, 17, 27, 14, 25, 56, 42, 30, 39,  8,  0, 28, 62])

In [81]:
# Menampilkan input dan prediksi karakter selanjutnya.
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'your youth,\nBut mine shall be a comfort to your age.\nThe loss you have is but a son being king,\nAnd '

Next Char Predictions:
 b"ri'sZ\n$K'n;.oyqTQHqMVRRTbiscId\nOpoc?\net CizRw&qGVg&Z?doGyoA,zU,iQCKA!H\nj3yf.$!Kc\nOQ!EYkJDNALqcQZ-[UNK]Ow"


## Train Model

**Menambahkan optimizer dan fungsi loss**

In [82]:
# loss function untuk pelatihan model yang digunakan untuk tugas seperti klasifikasi atau pemrosesan bahasa alami.
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [83]:
# Menghitung nilai kerugian (loss) model dengan fungsi SparseCategoricalCrossentropy.
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1888685, shape=(), dtype=float32)


In [84]:
# untuk menghitung eksponensial dari nilai rata-rata kerugian (mean loss) dalam bentuk tensor.
tf.exp(example_batch_mean_loss).numpy()

65.94813

In [85]:
# Menggunakan optimizer 'adam' dan loss yang telah dihitung sebelumnya.
model.compile(optimizer='adam', loss=loss)

**Konfigurasi Checkpoints**


In [86]:
# Menyiapkan direktori dan nama file checkpoint untuk menyimpan model.
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
# Membuat callback untuk menyimpan bobot model saat pelatihan.
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Melakukan proses Training**

In [87]:
EPOCHS = 10 # Menentukan jumlah EPOCHS (iterasi pelatihan)

In [88]:
# Memulai pelatihan model dengan dataset dan callback untuk menyimpan bobot.
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 13s 57ms/step - loss: 2.7047
Epoch 2/10
172/172 [==============================] - 11s 54ms/step - loss: 1.9889
Epoch 3/10
172/172 [==============================] - 11s 55ms/step - loss: 1.7093
Epoch 4/10
172/172 [==============================] - 11s 56ms/step - loss: 1.5468
Epoch 5/10
172/172 [==============================] - 12s 55ms/step - loss: 1.4476
Epoch 6/10
172/172 [==============================] - 11s 56ms/step - loss: 1.3796
Epoch 7/10
172/172 [==============================] - 11s 57ms/step - loss: 1.3280
Epoch 8/10
172/172 [==============================] - 12s 57ms/step - loss: 1.2818
Epoch 9/10
172/172 [==============================] - 12s 58ms/step - loss: 1.2414
Epoch 10/10
172/172 [==============================] - 12s 57ms/step - loss: 1.2025


## Generate Teks

In [89]:
# Mendefinisikan kelas OneStep yang bertindak sebagai model untuk menghasilkan teks karakter demi karakter.
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

   # Membuat masker untuk mencegah generasi karakter "[UNK]" (unknown) yang tidak diinginkan.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Memasang -inf pada setiap indeks yang tidak diinginkan.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # menyesuaikan bentuk dengan vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Mengonversi string menjadi token ID.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

   # Menjalankan model.
    # predicted_logits.shape adalah [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
   # Hanya menggunakan prediksi terakhir.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Menerapkan masker prediksi: mencegah "[UNK]" dari dihasilkan.
    predicted_logits = predicted_logits + self.prediction_mask

    # Mencoba output logits untuk menghasilkan token ID.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Mengonversi dari token ID menjadi karakter.
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Mengembalikan karakter dan status model.
    return predicted_chars, states

In [90]:
# Membuat instance dari model OneStep.
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [91]:
# Generasi teks karakter demi karakter dengan model OneStep.
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
# Kode diatas menghasilkan teks karakter demi karakter menggunakan model OneStep. Mulai dari karakter "ROMEO:", model menghasilkan karakter berikutnya dengan mengikuti probabilitas distribusi yang telah dipelajari selama pelatihan.

# Menampilkan hasil generasi teks.
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
why look their eyes?

First Lord:
Why, over hast thou with the welcome of the pardon.

YORK:
Even for a little, made me from your friends
And traitoble that more pain of our amils.

BAPTISTA:
Madam! if the else
A wime of Hereford sirs a grave Master cousin.

Second Ungains when he did prove
A sisterhood of this milder of them.

LUCENTIO:

LADY CAPULET:
No, sirt, a nobleman, Lord Northumberland,--
What nourish? 'Fail in person whether do loud could
Absence mides, our circles fools, as you
would not have your pettice on the head that yet quench you
To say to tell her counsellor mouth: tears barmands;
And if a man go town: he hath drait much controvery of it.
Thus live the durthes for resolved: their eyes hand so read
With one as you myselves faults, but as sound?

QUEEN MARGARET:
Ay, there is the pale and think the idle deaths
As well as so smolent of you this married breath,
'tis Richard and beat dreaming?

KING RICHARD III:
We are like the wrinkles with him!

SCANLECONT:
God gav

In [92]:
# Generasi teks karakter demi karakter dengan model OneStep yang telah dimuat kembali.
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

# Menampilkan hasil generasi teks dengan model yang telah dimuat kembali.
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nNurse, one that set of inclined, hither too\nmad poverty senses; as much given her uncles drown\nHim prosperous plant but of aften ease,\neven he have stood and Darentain lies\nAnd when I dare need no rearianing; the dutt\nbeen affection of a royal queen!\n\nFLORIZEL:\nI do; with him the chamory\nThe times of my sisters favour of their\na couched spite of right field.\n\nWARWICK:\nWas eye, my boy, let us swallow well they did:\nThe people is much patience, that also explecting embrace\nWhereof shall enter there is farnest usmes ne\nMartingare that sworn in sulf,\nHer title is already things and the world:\nGive diving yourself his wifes hoties: they all to die\nAs a time indeed to see a beforn.\n\nSecond Musician:\nTranio, thou aftice,\nThe hungry gape of all\nthe devil. How now! why come you of.\n\nSecond Murderer:\nThere's the best; and after two and the ord\nBut only so faster, I have with these season\nThe Volsces from your gamours' doubled to his\ncannot, the

 Perbedaan utama dengan kode sebelumnya adalah pola awal inputnya. Pada kode sebelumnya, pola awal adalah ROMEO:, sedangkan pada kode ini, pola awal adalah ROMEO: yang diulang sebanyak lima kali. Dengan demikian, dapat dilihat bahwa model yang telah dimuat kembali dapat digunakan untuk menghasilkan teks dari berbagai pola awal yang berbeda, dan hasilnya akan tergantung pada pola awal yang digunakan.

**Mengekspor Model Generator**

In [93]:
# Menyimpan model OneStep ke disk dengan nama 'one_step'.
tf.saved_model.save(one_step_model, 'one_step')
# Memuat model OneStep dari disk dengan nama 'one_step' yang telah disimpan sebelumnya.
one_step_reloaded = tf.saved_model.load('one_step')

In [94]:
# Menginisialisasi variabel states dan next_char.
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

# Melakukan generasi teks karakter demi karakter dengan model yang telah dimuat kembali.
for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

# Menampilkan hasil teks yang dihasilkan oleh model.
print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The white idwase assurate his folly!
The pacress spirition! of my thusb at you,
I cannot chysol exc


Output teks yang dihasilkan oleh model menunjukkan bahwa model telah belajar untuk menghasilkan teks yang konsisten dengan gaya dan bahasa drama Shakespeare.<br>
<br>
Secara keseluruhan, output teks tersebut menunjukkan bahwa model masih dalam pengembangan. Model telah belajar untuk menghasilkan teks yang konsisten dengan gaya dan bahasa Shakespeare, tetapi masih belum sempurna dalam menghasilkan teks yang realistis dan koheren.